### Reservoir compaction and subsidence

This script can be used to calculate the reservoir compaction and surface subsidence generated when depleting a reservoir. This python notebook follows the __[subsidence workflow document](https://statoilsrm.sharepoint.com/sites/RockMechanicsENGDCIRM/Shared%20Documents/Geomechanical%20modelling,%20DE4RM,%20etc/Best%20practice%20and%20workflow/Subsidence%20workflow.docx?web=1)__.

***

In [258]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import datetime
from datetime import datetime
import os.path
import ipywidgets as widgets
from ipywidgets import Box,TwoByTwoLayout,GridspecLayout,Button, Layout, interactive
import sys
import os
from IPython.display import display
import mpl_interactions.ipyplot as iplt
from IPython.display import display
import functools

In [2]:
# create input widgets
# set widget style
style = {'description_width':'initial'}

#widget for each user input
TR = widgets.BoundedFloatText(value=1734,min=0,max=10000,step=10,description='Top Reservoir (TVDMSL):',style=style,disabled=False)
BR = widgets.BoundedFloatText(value=1771,min=0,max=10000,step=10,description='Base Reservoir (TVDMSL):',style=style,disabled=False)
NGw = widgets.BoundedFloatText(value=0.88,min=0,max=1,step=0.1,description='NTG (fraction):',style=style,disabled=False)
WD = widgets.BoundedFloatText(value=127,min=0,max=10000,step=10,description='Water depth (m):',style=style,disabled=False)
RR = widgets.BoundedFloatText(value=3500,min=0,max=10000,step=100,description='Reservoir radius (m):',style=style,disabled=False)
D = widgets.BoundedFloatText(value=13.2,min=0,max=10000,step=10,description='Depletion (MPa):',style=style,disabled=False)
nuw = widgets.BoundedFloatText(value=0.29,min=0,max=1,step=0.01,description='Poissons Ratio (bar):',style=style,disabled=False)
Ew = widgets.BoundedFloatText(value=7.9,min=0,max=10000,step=1,description='Youngs Modulus (GPa):',style=style,disabled=False)
e = widgets.BoundedFloatText(value=8000,min=0,max=10000,step=1000,description='End of data range (m):',style=style,disabled=False)
intv = widgets.BoundedFloatText(value=100,min=0,max=1000,step=10,description='Data interval (m):',style=style,disabled=False)
ProjectName =  widgets.Text(value='Project1',placeholder='Type project name',description='Project Name:',style=style,disabled=False)
File = widgets.Text(value=r"C:\Users\dld\Downloads",placeholder='Type File Path',description='File Path:',style=style,disabled=False)

In [175]:
# function to calculate single variables (prints calculated variables)
def RT(TR,BR,NGw,WD,Ew,nuw,RR):
    RT = (BR-TR)*NGw
    OB = TR-WD
    K = Ew/(3*(1-2*nuw))
    C = OB/RR
    M = (Ew*(1-nuw))/((1-(2*nuw))*(1+nuw))
    B = 1-(K/37)
    Cm = 1/M/1000*B
    
    print('Reservoir thickness = {} m'.format("{:.2f}".format(RT))),
    print('Overburden thickness = {} m'.format("{:.2f}".format(OB))),
    print('Bulk Modulus, K = {} GPa'.format("{:.2f}".format(K))),
    print('Depth/radius ratio, C = {}'.format("{:.2f}".format(C))),
    print('Uniaxial compaction Modulus, M = {} GPa'.format("{:.2f}".format(M))),
    print('Biot coefficient = {}'.format("{:.2f}".format(B))),
    print('Compaction coefficient, Cm = {}'.format("{:0.2e}".format(Cm)))

# function to create a dataframe of all input variables (included calculated)
def variables(TR,BR,NGw,WD,Ew,nuw,RR,D):
    RT = (BR-TR)*NGw
    OB = TR-WD
    K = Ew/(3*(1-2*nuw))
    C = OB/RR
    M = (Ew*(1-nuw))/((1-(2*nuw))*(1+nuw))
    B = 1-(K/37)
    Cm = 1/M/1000*B
    
    Calc_variables = [['Top reservoir', TR,'mTVD'], 
              ['Base reservoir', BR,'mTVD'], 
              ['Reservoir thickness', RT,'m'], 
              ['N/G', NGw,'fraction'], 
              ['Water Depth', WD,'m'],
              ['OB thickness', OB,'m'],
              ['Reservoir radius', RR,'m'],
              ['C (depth/radius)', C,'fraction'],
              ['Depletion', D,'MPa'],
              ['Poissons ratio', nuw,''],
              ['Youngs Modulus, E', Ew,'GPa'],
              ['Bulk Modulus, K', K, 'GPa'],
              ['Uniaxial Compaction Modulus, M', M,'GPa'],
              ['Biot', B,''],
              ['Cm', Cm,'MPa^-1']]
    df_var = pd.DataFrame(Calc_variables, columns=['Input variable','value','units'])
    return df_var

In [255]:
# Main function to calculate geertsma compaction/subsidence
def geertsma(Base_res, Top_res, NG, Water_depth, E, nu, Reservoir_radius,e,intv, Depletion):
    Res_thickness = (Base_res-Top_res)*NG
    OB = Top_res-Water_depth
    K = E/(3*(1-2*nu))
    C = OB/Reservoir_radius
    M = (E*(1-nu))/((1-(2*nu))*(1+nu))
    B = 1-(K/37)
    Cm = 1/M/1000*B
    # create dataframe for a range of data points defined by the water depth (first depth) and depth of interest (number of data points controlled by input intv)
    df_results = pd.DataFrame((np.arange(Water_depth,e,intv)), columns=['Depth'])   
    df_results['Z (m)'] = ((df_results['Depth']) - Water_depth)/Top_res    # Creating normalised depth
    #creating conditions to asign data to OB, UB or reservoir
    conditions = [
        (df_results['Depth'] < Top_res),
        (df_results['Depth'] >= Top_res) & (df_results['Depth'] <= Base_res),
        (df_results['Depth'] > Base_res)
        ]
    df_results['definition'] = np.select(conditions, ['Overburden','Reservoir','Underburden'])   # defining data points as either overburden, reservoir and underburden
    # Calculation of Uz (broken into smaller componants)
    Z = df_results['Z (m)']
    a = (Cm/2)*Res_thickness*Depletion*B
    b = (C*(Z-1))/pow(1+C*C*(Z-1)*(Z-1),(1/2))
    c = ((3-(4*nu))*C*(Z+1))/pow(1+C*C*(Z+1)*(Z+1),(1/2))
    d = (2*C*Z)/pow((1+C*C*(Z+1)*(Z+1)),(3/2))
    e_OB = 3-(4*nu)+1   # +1 used if overburden
    e_UB = 3-(4*nu)-1   # -1 used if underburden
    conditions = [
        (df_results['definition'] == 'Overburden'),
        (df_results['definition'] == 'Reservoir'),
        (df_results['definition'] == 'Underburden')
        ]
    Uz_OB = a*(b-c+d+e_OB)
    Uz_UB = a*(b-c+d+e_UB)
    df_results['Uz (m)'] = np.select(conditions, [Uz_OB,Uz_OB,Uz_UB])   # calculation of Uz
    df_results['Uz (cm)'] = df_results['Uz (m)'] * 100
    Uz_end = df_results['Uz (m)'].iloc[-1]    # Extracting the end Uz value
    df_results['Vertical displacement (m)'] = df_results['Uz (m)'] - Uz_end    # Calculation of vertical displacement
    df_results['Vertical displacement (cm)'] = df_results['Vertical displacement (m)']*100

    df_maxD = df_results['Vertical displacement (cm)'].idxmax()     #finding index of row with max displacement - corresponds to top reservoir
    df_next = df_maxD + 1                                               #creating the index of next row after max displacement row
    df_results_compaction = df_results.loc[df_maxD:df_next]         # new dataframe with data at the top and base reservoir

    Vd_OB = df_results_compaction['Vertical displacement (cm)'].iloc[0]     # extracting the vertical displacement just above res
    Vd_UB = df_results_compaction['Vertical displacement (cm)'].iloc[-1]    # extracting the vertical displacement just beow the res
    Compaction = Vd_OB - Vd_UB   #calculating reservoir compaction
    Compaction_est = Res_thickness*Depletion*Cm*100 
    sb_sub = df_results.iloc[0,6]    # sea bed subsidence is the first displacement value calculated for seabed depth
    
    #outputs:
    print('Seabed subsidence: {:.1f} cm'.format(sb_sub))
    print('Reservoir compaction: {:.1f} cm'.format(Compaction))
    #print('Simplified reservoir compaction estimate: {:.1f} cm'.format(Compaction_est))
    return df_results

# creating interactive widget ouputs
out = widgets.interactive_output(RT, {'TR': TR,'BR': BR,'NGw':NGw,'WD':WD, 'Ew':Ew, 'nuw':nuw,'RR':RR})
out2 = widgets.interactive_output(geertsma, {'Base_res': BR,'Top_res': TR,'NG':NGw,'Water_depth':WD, 'E':Ew, 'nu':nuw,'Reservoir_radius':RR, 'e':e, 'intv':intv,'Depletion':D})



In [257]:
# allows function to be run while suppresing printed outputs
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# create a plot showing the vertical displacement profile that updates automatically when inputs are changed
%matplotlib inline
def plot(BR,TR,NGw,WD,Ew,nuw,RR,e,intv,D):
    # run function without print to get the results dataframe
    with HiddenPrints():
        df_results = geertsma(BR,TR,NGw,WD,Ew,nuw,RR,e,intv,D)
    
    # defining x-y data for sea bed and reservoir depth
    Vd_OB = df_results['Vertical displacement (cm)'].iloc[0]
    Vd_UB = df_results['Vertical displacement (cm)'].iloc[-1]
    x = [(Vd_UB-1),(Vd_OB+5)]
    Seabed_y = (WD,WD)
    Res_top_y = (TR,TR)
    Res_base_y = (BR, BR)
    
    # formatting plot
    f, ax = plt.subplots(figsize=(4,8))
    ax.set_ylim([0,e])   # setting y plot limit between 0 and depth of interest
    ax.set_xlim(x)   # setting x plot limit
    ax.axhline(y=0, color='darkgrey')    # Adding axis through origin
    ax.axvline(x=0, color='darkgrey')    # Adding axis through origin
    
    # plotting data
    ax.plot(x,Seabed_y,color='steelblue', label='Seabed')    # Plotting seabed level    
    ax.plot(x,Res_top_y,color='darkorange', label='Top reservoir')    # plotting top res
    ax.plot(x,Res_base_y,color='gold', label='Base reservoir')      # plotting base res
    ax.plot(df_results['Vertical displacement (cm)'], df_results['Depth'], color='k', label='displacement profile')   # plotting displacement profile
    
    # formatting plot/data
    ax.invert_yaxis()
    ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
    ax.set_title('Vertical Displacement Profile')
    ax.set_xlabel('Vertical displacement (cm)')
    ax.set_ylabel('Depth (TVDm)')
    ax.xaxis.set_label_position('top')
    ax.legend()
    plt.show(block=False)

# creating interactive output widget that shows the plot
out3 = widgets.interactive_output(plot,{'BR':BR,'TR':TR,'NGw':NGw,'WD':WD,'Ew':Ew,'nuw':nuw,'RR':RR,'e':e,'intv':intv,'D':D })


In [243]:
# HTML widgets
HTML = widgets.HTML(value = '<br> <big> <b>Directory for exporting data <b>')
HTML2 = widgets.HTML(value = 'Calculated data is exported as a .xlsx file to chosen directory')
HTML3 = widgets.HTML(value = '<h1> Geertsma subsidence/compaction estimate <h1>')
HTML4 = widgets.HTML(value = '<br> <big> <b>User inputs <b>')
HTML5 = widgets.HTML(value = '<br> <big> <b>Calculated variables <b>')
HTML6 = widgets.HTML(value = '<br> <b>Define data limits <b>')
HTML7 = widgets.HTML(value = '<br> <big><b>Show calculated data table <b>')
HTML8 = widgets.HTML(value = '<hr>')

In [167]:
# create button to save data
Savebutton = widgets.Button(description="Download data for current inputs",button_style='info', layout=Layout(width='97%'))
output = widgets.Output()

# function to run when button is clicked - if clicked it will save data to an excel document with two sheets (one with the inputs and one with the results dataframe)
def on_button_clicked(b):
    with output:
        output.clear_output()
        with HiddenPrints(): 
            df_results = geertsma(BR.value,TR.value,NGw.value,WD.value,Ew.value,nuw.value,RR.value,e.value,intv.value,D.value)
            df_variables = variables(TR.value,BR.value,NGw.value,WD.value,Ew.value,nuw.value,RR.value,D.value)
        date = datetime.now().strftime("%d_%m_%Y_%H-%M-%S")  
        
        # define file path - appends, the input from the file path widget, the project name and the date&time
        file_name = '{}\Subsidence_{}_{}.xlsx'.format(File.value,ProjectName.value,date)

        # check that file doesn't already exist
        check_file = os.path.isfile(file_name)

        # if function - will save data if file of assigned name doesn't exist. will throw error message to change project name otherwise
        if check_file == True:
            print('File already exists with this name, give new project name')
        else:
            with pd.ExcelWriter(file_name) as writer:
                df_variables.to_excel(writer, sheet_name='Variables', index=False)
                df_results.to_excel(writer, sheet_name='Data tables',index=False)
            print('Results exported to: {}'.format(file_name))

# run the on_button_clicked function when button is clicked
Savebutton.on_click(functools.partial(on_button_clicked, \
                                 ))


In [241]:
# Checkbox widget gives the option to show the results dataframe
Showdata = widgets.Checkbox(value=False,description='Tick to show data',disabled=False,indent=False)

# function that runs with checkbox output
def show(Showdata):
    with HiddenPrints(): 
            df_results = geertsma(BR.value,TR.value,NGw.value,WD.value,Ew.value,nuw.value,RR.value,e.value,intv.value,D.value)
    # format so that whole dataframe is displayed
    with pd.option_context('display.max_rows',None,
                           'display.max_columns', None,
                           'display.precision',2):
        if Showdata:
            display(df_results)
        else:
            print('')

#interactive widget
out7 = widgets.interactive_output(show,{'Showdata':Showdata})

In [256]:
# display the interactive widgets to calculate subsidence and compaction
widgets.HBox([widgets.VBox([HTML4,ProjectName,TR, BR, NGw, WD,RR,D,Ew,nuw,HTML6,e,intv, HTML5,out,HTML7, out2, HTML,HTML2, File,Savebutton,output]), widgets.VBox([HTML7,Showdata,HTML8,out3]),out7])